In [1]:
import nibabel as nib
import numpy as np
import pandas as pd
import os 

In [3]:
# Read the .txt file into a DataFrame
atlas_data = pd.read_csv("receptors/AAL_90_labels.txt", sep="\t", header=None)
# Assign column names
atlas_data.columns = ["code", "label_name", "roi_label"]
receptor_fnames = [ '5HT2a_cimbi_hc29_beliveau',
 '5HT1a_cumi_hc8_beliveau',
 '5HT6_gsk_hc30_radhakrishnan',
 'D2_flb457_hc37_smith',     
 'D1_SCH23390_hc13_kaller',       
    'H3_cban_hc8_gallezot', 
 'MU_carfentanil_hc204_kantonen',
 'DAT_fpcit_hc174_dukart_spect']
dataframes_dict = {}
for fname in receptor_fnames:
    receptor_file = os.path.join("receptors", fname + ".csv")
    receptor_parc = np.genfromtxt(receptor_file, delimiter=',')[:90]
    receptor_orig = receptor_parc / ((np.max(receptor_parc)) - np.min(receptor_parc))
    receptor_norm = receptor_orig - np.max(receptor_orig) + 1
    receptor_norm[receptor_norm < 0] = 0 
    # Create DataFrame with additional column 'normalized_values'
    df = atlas_data.copy()
    df['normalized_value'] = receptor_norm
    df['original_value'] = receptor_parc
    # Store DataFrame in the dictionary with fname as the key
    dataframes_dict[fname] = df

In [4]:
import nibabel as nib
import numpy as np


def modify_atlas(atlas_path, atlas_roi_labels, marker_values, atlas_name=None):
    """
    Modify the values in an atlas image.
    marker_values is a dataframe with the following columns:
    - roi_label: the label of the ROI in the atlas
    - normalized_value: the new value to assign to the ROI
    """
    # Load the atlas image

    atlas_img = nib.load(atlas_path)


    # Initialize a matrix with the same shape as the atlas
    combined_matrix = np.zeros(atlas_img.shape)
    recognized_labels = 0
    for label in atlas_roi_labels:
        # Check if the label exists in marker_values
        if label in marker_values['roi_label'].values:
            # From the dataframe marker_values, get the normalized_value which corresponds to the current label
            new_val = marker_values[marker_values['roi_label'] == label]['normalized_value'].values[0]
            recognized_labels+=1
        else:
            print(f"Label {label} not found in marker_values")

        # Create a binary mask for the current ROI using the atlas image
        from nilearn import image
        idx_map = image.math_img('img == %s' % label, img=atlas_path)
        idx_map_data = idx_map.get_fdata()

        # Set the marker value for voxels within the current ROI
        idx_map_data[idx_map_data != 0] = new_val
        print(f'Unique values in idx_map : {np.unique(idx_map_data)}')
        
        # Add the current ROI mask to the combined matrix
        combined_matrix = np.add(combined_matrix, idx_map_data)

    # Create a new Nifti image with the modified data
    modified_atlas_img = nib.Nifti1Image(combined_matrix, atlas_img.affine, atlas_img.header)
    # or: 
    # modified_atlas_img = new_img_like(atlas_img, combined_matrix)
    
    return modified_atlas_img

In [6]:
dataframes_dict['5HT1a_cumi_hc8_beliveau']

,code,label_name,roi_label,normalized_value,original_value
0,FAG,Precentral_L,2001,0.433605,25.700185
1,FAD,Precentral_R,2002,0.405310,24.219469
2,F1G,Frontal_Sup_L,2101,0.517487,30.089834
3,F1D,Frontal_Sup_R,2102,0.502637,29.312744
4,F1OG,Frontal_Sup_Orb_L,2111,0.637951,36.393872
...,...,...,...,...,...
85,T2D,Temporal_Mid_R,8202,0.698504,39.562695
86,T2AG,Temporal_Pole_Mid_L,8211,1.000000,55.340355
87,T2AD,Temporal_Pole_Mid_R,8212,0.914360,50.858731
88,T3G,Temporal_Inf_L,8301,0.782781,43.972997


In [ ]:
import matplotlib.pyplot as plt
for idx, receptor_fname in enumerate(receptor_fnames[:4]):
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

    axs[0].hist(dataframes_dict[receptor_fname]['original_value'], bins=20)
  
    axs[0].set_xlabel("Original density", fontsize=24)
    axs[0].set_ylabel("Count", fontsize=20)
   
    axs[0].tick_params(axis='x', labelsize=18)
    axs[0].tick_params(axis='y', labelsize=18)
   
    axs[1].hist(dataframes_dict[receptor_fname]['normalized_value'], bins=20)
   
    axs[1].set_xlabel("Normalized density", fontsize=24)
    axs[1].set_ylabel("Count", fontsize=20)
    axs[1].tick_params(axis='x', labelsize=18)
    axs[1].tick_params(axis='y', labelsize=18)
    
    plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
for idx, receptor_fname in enumerate(receptor_fnames[4:]):
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    
    axs[0].hist(dataframes_dict[receptor_fname]['original_value'], bins=20)
   
    axs[0].set_xlabel("Original density", fontsize=24)
    axs[0].set_ylabel("Count", fontsize=20)
   
    axs[0].tick_params(axis='x', labelsize=18)
    axs[0].tick_params(axis='y', labelsize=18)
    
    axs[1].hist(dataframes_dict[receptor_fname]['normalized_value'], bins=20)
   
    axs[1].set_xlabel("Normalized density", fontsize=24)
    axs[1].set_ylabel("Count", fontsize=20)
    axs[1].tick_params(axis='x', labelsize=18)
    axs[1].tick_params(axis='y', labelsize=18)
   
    plt.tight_layout()

In [ ]:
from nilearn import plotting
atlas_roi_labels = atlas_data['roi_label']
for fname in receptor_fnames:
      receptor_image = modify_atlas("receptors/AAL_90.nii.gz", atlas_roi_labels, dataframes_dict[fname])
      # Plot the modified atlas using an inversed colormap making darkers a higher value
      display = plotting.plot_glass_brain(stat_map_img=receptor_image,
                                    colorbar=True,
                                    cmap='viridis',
                                    alpha=1,
                                    display_mode="lyrz")
      display.add_contours(receptor_image, filled=False)
      recp_mean = dataframes_dict[fname]['normalized_value'].mean()
      display.title(fname.split('_')[0]+ f" | Mean: {recp_mean:.2f}")
     